In [1]:
import gumpy as gp
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import sklearn
import os


import warnings
import time
warnings.simplefilter('ignore') #忽略警告

In [2]:
import scipy
import scipy.io as sio

import scipy.signal

from scipy import linalg

import pandas as pd

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
#分类器
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.lda import LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

import xgboost
import lightgbm


#模型调节
from mlxtend.feature_selection import SequentialFeatureSelector #特征选择函数 选择合适的feature

#结果可视化
from sklearn.metrics import classification_report , confusion_matrix #混淆矩阵

#相关指标
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from sklearn.metrics import cohen_kappa_score

from sklearn.metrics import roc_auc_score

#二分类其多分类化
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OneVsRestClassifier

#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans

#距离函数 度量向量距离
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity #余弦相似度

#one-hot使用
#from keras.utils import to_categorical

from sklearn.preprocessing import label_binarize

#绘图
import matplotlib.pyplot as plt

import scipy.linalg as la

import gc

%matplotlib inline

In [3]:
from statsmodels.tsa.ar_model import AR #自回归模型
from python_speech_features import mfcc as pmfcc

In [4]:
sample_rate = 256 #hz
origin_channel = 16 #5 channel eeg

#采集的通道
#共16 channel
#未使用的channel使用none代替
#reference:a study on performance increasing in ssvep based bci application
SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

# 减去前多少秒数据 second
# 减去后多少秒数据 second
CLIP_FORWARD = 1
CLIP_BACKWARD = 1

# 单个小段的实验时长
trial_time = 3 #second

trial_offset = 0 #second
start_trial_time = 0 #真正的实验开始时刻
end_trial_time = 2 #真正的实验结束时刻(<trial_time)

#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频



In [5]:
def load_data(filename):
    
    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix
    
    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate] #首部 尾部 进行裁剪
    
    return data

In [6]:
def separate(data , label , overlap_length = 128):
    '''
    最长重叠长度为size长 256*3 个数据点
    '''
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0

    while idx<data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)

    return np.array(train_data) , np.array(train_labels)

In [7]:
def train_val(data , ratio = 0.9):
    '''
    将数据分为 训练集 和 验证集
    '''
    
    seg = int(ratio * data.shape[0])
    
    return data[ : seg] , data[seg : ]

def shuffle_t_v(filenames):
    # np.random.shuffle(filenames)
    
    return np.random.choice(filenames , size=10) #20次的计算准确率中 每次随机选择10个样本进行训练测试

def combine(freq):
    '''
    训练数据与验证数据
    :freq: 指定闪烁的频率
    
    '''
    
    if freq not in [6 , 7.5 , 8.5 , 10]:
        print('freq must in {6,7.5,8.5,10}')
        return 
    
    ratio = 0.9 #训练集的占比
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('circle/0/%s/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('circle/1/%s/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('circle/2/%s/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('circle/3/%s/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('circle/4/%s/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('circle/5/%s/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('circle/6/%s/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('circle/7/%s/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('circle/8/%s/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('circle/0/%s/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('circle/1/%s/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('circle/2/%s/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('circle/3/%s/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('circle/4/%s/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('circle/5/%s/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('circle/6/%s/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('circle/7/%s/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('circle/8/%s/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    
    person_0_train , person_0_val = train_val(person_0)
    person_1_train , person_1_val = train_val(person_1)
    person_2_train , person_2_val = train_val(person_2)
    person_3_train , person_3_val = train_val(person_3)
    person_4_train , person_4_val = train_val(person_4)
    person_5_train , person_5_val = train_val(person_5)
    person_6_train , person_6_val = train_val(person_6)
    person_7_train , person_7_val = train_val(person_7)
    person_8_train , person_8_val = train_val(person_8)
    
    #数据分段阶段
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0_train , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1_train , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2_train , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3_train , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4_train , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5_train , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6_train , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7_train , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8_train , label = 8 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                    ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
    
    #============
    #验证数据分段
    val_person_data_0 , val_person_labels_0 = separate(person_0_val , label = 0 , overlap_length=0)
    val_person_data_1 , val_person_labels_1 = separate(person_1_val , label = 1 , overlap_length=0)
    val_person_data_2 , val_person_labels_2 = separate(person_2_val , label = 2 , overlap_length=0)
    val_person_data_3 , val_person_labels_3 = separate(person_3_val , label = 3 , overlap_length=0)
    val_person_data_4 , val_person_labels_4 = separate(person_4_val , label = 4 , overlap_length=0)
    val_person_data_5 , val_person_labels_5 = separate(person_5_val , label = 5 , overlap_length=0)
    val_person_data_6 , val_person_labels_6 = separate(person_6_val , label = 6 , overlap_length=0)
    val_person_data_7 , val_person_labels_7 = separate(person_7_val , label = 7 , overlap_length=0)
    val_person_data_8 , val_person_labels_8 = separate(person_8_val , label = 8 , overlap_length=0)
    
    #合并数据
    val_data = np.concatenate((val_person_data_0 , val_person_data_1 , val_person_data_2 ,
                               val_person_data_3 , val_person_data_4 , val_person_data_5 ,
                               val_person_data_6 , val_person_data_7 , val_person_data_8 ,
                               ))
    
    val_labels = np.concatenate((val_person_labels_0 , val_person_labels_1 , val_person_labels_2 ,
                                 val_person_labels_3 , val_person_labels_4 , val_person_labels_5 ,
                                 val_person_labels_6 , val_person_labels_7 , val_person_labels_8 ,
                                 ))
    
    #产生索引并置乱
    idx_val_data = list(range(val_data.shape[0]))
    np.random.shuffle(idx_val_data)

    #将训练数据置乱
    val_data = val_data[idx_val_data]
    val_labels = val_labels[idx_val_data]
    
    return train_data , train_labels , val_data , val_labels

def shuffle(data , labels):
    idx_data = list(range(data.shape[0]))
    np.random.shuffle(idx_data)

    data = data[idx_data]
    labels = labels[idx_data]
    
    return data , labels

# SBP

In [8]:
def butter_worth(data , lowcut , highcut , order=6):
    nyq = 0.5 * sample_rate
    
    lo = lowcut / nyq
    hi = highcut / nyq
    
    b,a = scipy.signal.butter(order , [lo , hi] , btype='bandpass')

    return np.array([scipy.signal.filtfilt(b , a , data[: , i]) for i in range(data.shape[1])]).reshape((-1 , origin_channel))

def alpha_subBP_features(data):
    alpha1 = butter_worth(data , 8.5 , 11.5)
    alpha2 = butter_worth(data , 9.0 , 12.5)    
    alpha3 = butter_worth(data , 9.5 , 13.5)   #11.5 后
    alpha4 = butter_worth(data , 8.0 , 10.5)   
    
    return np.array([alpha1 , alpha2 , alpha3 , alpha4])

def beta_subBP_features(data):
    beta1 = butter_worth(data , 15.0 , 30.0) #14.0 前
    beta2 = butter_worth(data , 16.0 , 17.0)    
    beta3 = butter_worth(data , 17.0 , 18.0)    
    beta4 = butter_worth(data , 18.0 , 19.0)    
    
    return np.array([beta1 , beta2 , beta3 , beta4])

def powermean(data):
    #官方demo跳4秒 前4秒为准备阶段
    return np.power(data[ : , 0] , 2).mean(), \
            np.power(data[ : , 1] , 2).mean(), \
            np.power(data[ : , 2] , 2).mean(), \
            np.power(data[ : , 3] , 2).mean(), \
            np.power(data[ : , 4] , 2).mean(), \
            np.power(data[ : , 5] , 2).mean(), \
            np.power(data[ : , 6] , 2).mean(), \
            np.power(data[ : , 7] , 2).mean(), \
            np.power(data[ : , 8] , 2).mean(), \
            np.power(data[ : , 9] , 2).mean(), \
            np.power(data[ : , 10] , 2).mean(), \
            np.power(data[ : , 11] , 2).mean(), \
            np.power(data[ : , 12] , 2).mean(), \
            np.power(data[ : , 13] , 2).mean(), \
            np.power(data[ : , 14] , 2).mean(), \
            np.power(data[ : , 15] , 2).mean()       
            
def log_subBP_feature_extraction(alpha , beta):
    #alpha
    power_1_a = powermean(alpha[0])
    power_2_a = powermean(alpha[1])
    power_3_a = powermean(alpha[2])
    power_4_a = powermean(alpha[3])
    
    #beta
    power_1_b = powermean(beta[0])
    power_2_b = powermean(beta[1])
    power_3_b = powermean(beta[2])
    power_4_b = powermean(beta[3])
    
    X= np.array(
        [np.log(power_1_a) ,
         np.log(power_2_a) ,
         np.log(power_3_a) ,
         np.log(power_4_a) ,
         np.log(power_1_b) ,
         np.log(power_2_b) ,
         np.log(power_3_b) ,
         np.log(power_4_b)
        ]
        ).flatten()

    return X


def feature_extraction_sub_band_power(data):
    n_features = 128
    X = np.zeros((data.shape[0] , n_features))
    
    for i , datum in enumerate(data):
        alpha = alpha_subBP_features(datum)
        beta = beta_subBP_features(datum)
            
        X[i, :] = log_subBP_feature_extraction(alpha , beta)

    return X

# MFCC

In [9]:
def mfcc(data):
    
    X = []
   
    for i in range(16):
        X.append( pmfcc( data[: , i] , samplerate=256).reshape([1 , -1]).squeeze() )
        
    return np.array( X ).reshape([1 , -1]).squeeze()

def feature_extraction_mfcc(data):
    X = []
    
    for datum in data:
        X.append( mfcc(datum) )

    return np.array(X)

# AR

In [10]:
def ar(data):
    X = []
    
    for i in range(16):
        X.append( AR(data[: , i]).fit().params.reshape([1 , -1]).squeeze() )
        
    return np.array(X).reshape([1 , -1]).squeeze()

def feature_extraction_ar(data):
    X = []
    
    for datum in data:
        X.append( ar(datum) )
    
    return np.array(X)

In [11]:
train_X_ , train_y , val_X_ , val_y = combine(freq = 6) #6 7.5 8.5 10 hz
print(train_X_.shape , train_y.shape , val_X_.shape , val_y.shape)


(2241, 768, 16) (2241,) (81, 768, 16) (81,)


In [12]:
train_X = feature_extraction_mfcc(train_X_)
val_X = feature_extraction_mfcc(val_X_)

In [ ]:
print(train_X.shape , train_y.shape , val_X.shape , val_y.shape)

(2241, 53040) (2241,) (81, 53040) (81,)


In [68]:
#PCA降维
pca = PCA(n_components=256)
pca.fit(train_X)

train_X = pca.transform(train_X)
val_X = pca.transform(val_X)

In [27]:
#LDA降维
lda = LinearDiscriminantAnalysis(n_components=512)
lda.fit(train_X , train_y)

train_X = lda.transform(train_X)
val_X = lda.transform(val_X)

In [ ]:
#tSNE降维
#LDA降维
tsne = TSNE(n_components=256)
tsne.fit(train_X , train_y)

train_X = tsne.transform(train_X)
val_X = tsne.transform(val_X)

In [29]:
#不行
lgbm = lightgbm.LGBMClassifier()
lgbm.fit(train_X , train_y)
lgbm.score(val_X , val_y)

0.6666666666666666

In [30]:
#不行
xgb = xgboost.XGBClassifier()
xgb.fit(train_X , train_y)
xgb.score(val_X , val_y)

0.5432098765432098

In [31]:
#不行
gbdt = GradientBoostingClassifier()
gbdt.fit(train_X , train_y)
gbdt.score(val_X , val_y)

0.5802469135802469

In [32]:
rf = RandomForestClassifier()
rf.fit(train_X , train_y)
rf.score(val_X , val_y)

0.5679012345679012

In [33]:
#不行
svc = SVC(probability=True)
svc.fit(train_X , train_y)
svc.score(val_X , val_y)

0.4444444444444444

In [34]:
knn = KNeighborsClassifier()
knn.fit(train_X , train_y)
knn.score(val_X , val_y)

0.6419753086419753

In [35]:
nb = GaussianNB()
nb.fit(train_X , train_y)
nb.score(val_X , val_y)

0.5432098765432098

In [ ]:
classifiers = [lightgbm.LGBMClassifier , xgboost.XGBClassifier , GradientBoostingClassifier , RandomForestClassifier ,\
               SVC , KNeighborsClassifier , GaussianNB]

for freq in [6 , 7.5 , 8.5 , 10]:
    for idx , classifier in enumerate(classifiers):
        
        all_times = []
        all_accus = []
        
        for i in range(20):
            start = time.clock() #开始计时

            train_X_ , train_y , val_X_ , val_y = combine(freq = freq) #6 7.5 8.5 10 hz
            
            train_X = feature_extraction_sub_band_power(train_X_)
            val_X = feature_extraction_sub_band_power(val_X_)
            
            cls = classifier()
            cls.fit(train_X , train_y)
            score = cls.score(val_X , val_y)
            
            elapsed = (time.clock() - start) #终止计时并获得时长
            
            all_times.append(elapsed)
            all_accus.append(score)
            
        print('freq:%f classifier:%d avg_times:%f avg_accus:%f' % (freq , idx , np.average(all_times) , np.average(all_accus)))

freq:6.000000 classifier:0 avg_times:47.729483 avg_accus:0.733951
freq:6.000000 classifier:1 avg_times:53.301252 avg_accus:0.700617
freq:6.000000 classifier:2 avg_times:60.414503 avg_accus:0.732716
freq:6.000000 classifier:3 avg_times:46.607525 avg_accus:0.677778
freq:6.000000 classifier:4 avg_times:46.750078 avg_accus:0.620988
freq:6.000000 classifier:5 avg_times:46.247913 avg_accus:0.617284
freq:6.000000 classifier:6 avg_times:46.388816 avg_accus:0.259259
freq:7.500000 classifier:0 avg_times:57.321772 avg_accus:0.720988
freq:7.500000 classifier:1 avg_times:56.974473 avg_accus:0.707407
freq:7.500000 classifier:2 avg_times:58.069407 avg_accus:0.732716
freq:7.500000 classifier:3 avg_times:43.764980 avg_accus:0.685185
freq:7.500000 classifier:4 avg_times:44.119794 avg_accus:0.661111
freq:7.500000 classifier:5 avg_times:43.725553 avg_accus:0.603086
freq:7.500000 classifier:6 avg_times:43.624124 avg_accus:0.237654
freq:8.500000 classifier:0 avg_times:54.658695 avg_accus:0.761111
freq:8.500

In [11]:
classifiers = [lightgbm.LGBMClassifier , xgboost.XGBClassifier , GradientBoostingClassifier , RandomForestClassifier ,\
               SVC , KNeighborsClassifier , GaussianNB]

for freq in [6 , 7.5 , 8.5 , 10]:
    for idx , classifier in enumerate(classifiers):
        
        all_times = []
        all_accus = []
        
        for i in range(20):
            start = time.clock() #开始计时

            train_X_ , train_y , val_X_ , val_y = combine(freq = freq) #6 7.5 8.5 10 hz
            
            train_X = feature_extraction_mfcc(train_X_)
            val_X = feature_extraction_mfcc(val_X_)
            
            #训练PCA
            pca = PCA(n_components=256)
            pca.fit(train_X)
            
            train_X = pca.transform(train_X)
            val_X = pca.transform(val_X)
            
            cls = classifier()
            cls.fit(train_X , train_y)
            score = cls.score(val_X , val_y)
            
            elapsed = (time.clock() - start) #终止计时并获得时长
            
            all_times.append(elapsed)
            all_accus.append(score)
            
        print('freq:%f classifier:%d avg_times:%f avg_accus:%f' % (freq , idx , np.average(all_times) , np.average(all_accus)))

freq:6.000000 classifier:0 avg_times:138.282516 avg_accus:0.711111
freq:6.000000 classifier:1 avg_times:141.269782 avg_accus:0.726543
freq:6.000000 classifier:2 avg_times:146.849259 avg_accus:0.740741
freq:6.000000 classifier:3 avg_times:119.093152 avg_accus:0.694444
freq:6.000000 classifier:4 avg_times:122.441336 avg_accus:0.401852
freq:6.000000 classifier:5 avg_times:119.057277 avg_accus:0.735802
freq:6.000000 classifier:6 avg_times:120.345698 avg_accus:0.432716
freq:7.500000 classifier:0 avg_times:158.640500 avg_accus:0.816667
freq:7.500000 classifier:1 avg_times:138.361899 avg_accus:0.776543
freq:7.500000 classifier:2 avg_times:142.713647 avg_accus:0.781481
freq:7.500000 classifier:3 avg_times:114.939179 avg_accus:0.752469
freq:7.500000 classifier:4 avg_times:118.310052 avg_accus:0.443210
freq:7.500000 classifier:5 avg_times:115.028141 avg_accus:0.791975
freq:7.500000 classifier:6 avg_times:115.089906 avg_accus:0.419136
freq:8.500000 classifier:0 avg_times:150.309882 avg_accus:0.75

In [11]:
classifiers = [lightgbm.LGBMClassifier , xgboost.XGBClassifier , GradientBoostingClassifier , RandomForestClassifier ,\
               SVC , KNeighborsClassifier , GaussianNB]

for freq in [6 , 7.5 , 8.5 , 10]:
    for idx , classifier in enumerate(classifiers):
        
        all_times = []
        all_accus = []
        
        for i in range(20):
            start = time.clock() #开始计时

            train_X_ , train_y , val_X_ , val_y = combine(freq = freq) #6 7.5 8.5 10 hz
            
            train_X = feature_extraction_ar(train_X_)
            val_X = feature_extraction_ar(val_X_)
            
            cls = classifier()
            cls.fit(train_X , train_y)
            score = cls.score(val_X , val_y)
            
            elapsed = (time.clock() - start) #终止计时并获得时长
            
            all_times.append(elapsed)
            all_accus.append(score)
            
        print('freq:%f classifier:%d avg_times:%f avg_accus:%f' % (freq , idx , np.average(all_times) , np.average(all_accus)))

freq:6.000000 classifier:0 avg_times:134.992096 avg_accus:0.930864
freq:6.000000 classifier:1 avg_times:147.874691 avg_accus:0.929630
freq:6.000000 classifier:2 avg_times:150.530140 avg_accus:0.909259
freq:6.000000 classifier:3 avg_times:113.705355 avg_accus:0.895679
freq:6.000000 classifier:4 avg_times:117.483545 avg_accus:0.438889
freq:6.000000 classifier:5 avg_times:117.254820 avg_accus:0.584568
freq:6.000000 classifier:6 avg_times:118.716282 avg_accus:0.502469
freq:7.500000 classifier:0 avg_times:163.936999 avg_accus:0.904321
freq:7.500000 classifier:1 avg_times:143.222265 avg_accus:0.930247
freq:7.500000 classifier:2 avg_times:151.596896 avg_accus:0.887037
freq:7.500000 classifier:3 avg_times:117.628849 avg_accus:0.892593
freq:7.500000 classifier:4 avg_times:116.416074 avg_accus:0.385185
freq:7.500000 classifier:5 avg_times:113.121343 avg_accus:0.630247
freq:7.500000 classifier:6 avg_times:113.059178 avg_accus:0.467284
freq:8.500000 classifier:0 avg_times:155.864547 avg_accus:0.94